In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import geopy.distance
from math import radians,cos,sin,asin,sqrt
from scipy.stats import ttest_ind

# Choropleth (advanced)

In [2]:
pip install geojson

In [3]:
import h3
import json
import folium
from geojson.feature import *
import branca.colormap as cm 
from folium import Map, Marker, GeoJson 

import warnings
warnings.filterwarnings('ignore')

In [4]:
pip install h3

In [5]:
import os

In [6]:
os.listdir(r"/Users/ilkaymueller/Documents/GitHub/Data_Analysis_Projects/uber_files")

['Choropleth map.ipynb',
 '.DS_Store',
 'other-Lyft_B02510.csv',
 'Uber- NYC.ipynb',
 'other-FHV-services_jan-aug-2015.csv',
 'other-Firstclass_B01536.csv',
 'other-Skyline_B00111.csv',
 'uber-raw-data-janjune-15_sample.csv',
 'uber-raw-data-janjune-15.csv',
 'other-American_B01362.csv',
 'uber-raw-data-apr14.csv',
 'Uber-Jan-Feb-FOIL.csv',
 'other-Highclass_B01717.csv',
 'uber-raw-data-aug14.csv',
 'uber-raw-data-sep14.csv',
 'uber-raw-data-jul14.csv',
 'README.md',
 'other-Federal_02216.csv',
 'uber-raw-data-jun14.csv',
 'other-Carmel_B00256.csv',
 'other-Diplo_B01196.csv',
 '.ipynb_checkpoints',
 'other-Dial7_B00887.csv',
 'uber-raw-data-may14.csv',
 'other-Prestige_B01338.csv']

In [7]:
uber_data_apr14 = pd.read_csv("/Users/ilkaymueller/Documents/GitHub/Data_Analysis_Projects/uber_files/uber-raw-data-apr14.csv")

In [8]:
uber_data_apr14

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512
...,...,...,...,...
564511,4/30/2014 23:22:00,40.7640,-73.9744,B02764
564512,4/30/2014 23:26:00,40.7629,-73.9672,B02764
564513,4/30/2014 23:31:00,40.7443,-73.9889,B02764
564514,4/30/2014 23:32:00,40.6756,-73.9405,B02764


In [9]:
uber_data_apr14['Date/Time'] = uber_data_apr14['Date/Time'].astype('datetime64[ns]')  

In [10]:
print("Date from : {}".format(uber_data_apr14['Date/Time'].min()))
print("Date to   : {}".format(uber_data_apr14['Date/Time'].max())) 

Date from : 2014-04-01 00:00:00
Date to   : 2014-04-30 23:59:00


In [11]:
resolution = 8 

df = uber_data_apr14[["Lat", "Lon"]] 

df["hex_id"] = df.apply(lambda row: h3.geo_to_h3(row["Lat"], row["Lon"], resolution), axis = 1) 

df_aggreg = df.groupby(by = "hex_id").size().reset_index() 
df_aggreg.columns = ["hex_id", "value"] 

df_aggreg["geometry"] =  df_aggreg.hex_id.apply(lambda x: {"type" : "Polygon",
                                                           "coordinates": [h3.h3_to_geo_boundary(h=x,geo_json=True)]})

In [12]:
df_aggreg.sort_values(by = "value", ascending = False, inplace = True)  


In [13]:
def hexagons_dataframe_to_geojson(df_hex, file_output = None, column_name = "value"):
    """
    Produce the GeoJSON for a dataframe, constructing the geometry from the "hex_id" column
    and with a property matching the one in column_name
    """    
    list_features = []
    
    for i,row in df_hex.iterrows():
        try:
            geometry_for_row = { "type" : "Polygon", "coordinates": [h3.h3_to_geo_boundary(h=row["hex_id"],geo_json=True)]}
            feature = Feature(geometry = geometry_for_row , id=row["hex_id"], properties = {column_name : row[column_name]})
            list_features.append(feature)
        except:
            print("An exception occurred for hex " + row["hex_id"]) 
    feat_collection = FeatureCollection(list_features)
    geojson_result = json.dumps(feat_collection)
    return geojson_result

In [14]:
def choropleth_map(df_aggreg, border_color = 'black', fill_opacity = 0.7, initial_map = None, with_legend = False,
                   kind = "linear"):   
    #colormap
    min_value = df_aggreg["value"].min()
    max_value = df_aggreg["value"].max()
    m = round ((min_value + max_value ) / 2 , 0)
    
    #take resolution from the first row
    res = h3.h3_get_resolution(df_aggreg.loc[0, 'hex_id'])
    
    if initial_map is None:
        initial_map = Map(location= [40.7128, -74.0060], zoom_start=10, tiles="cartodbpositron", 
                attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>' 
            )
        

    #the colormap 
    #color names accepted https://github.com/python-visualization/branca/blob/master/branca/_cnames.json
    if kind == "linear":
        custom_cm = cm.LinearColormap(['green','yellow','red'], vmin=min_value, vmax=max_value)
    elif kind == "outlier":
        #for outliers, values would be -11,0,1
        custom_cm = cm.LinearColormap(['blue','white','red'], vmin=min_value, vmax=max_value)
    elif kind == "filled_nulls":
        custom_cm = cm.LinearColormap(['sienna','green','yellow','red'], 
                                      index=[0,min_value,m,max_value],vmin=min_value,vmax=max_value)
   

    #create geojson data from dataframe
    geojson_data = hexagons_dataframe_to_geojson(df_hex = df_aggreg)
    
    #plot on map
    name_layer = "Choropleth " + str(res)
    if kind != "linear":
        name_layer = name_layer + kind
        
    GeoJson(
        geojson_data,
        style_function = lambda feature: {
            'fillColor': custom_cm(feature['properties']['value']),
            'color': border_color,
            'weight': 1,
            'Highlight': True,
            'fillOpacity': fill_opacity 
        }, 
        name = name_layer
    ).add_to(initial_map)
    
    #add legend (not recommended if multiple layers)
    if with_legend == True:
        custom_cm.add_to(initial_map)   
    
    
    return initial_map

In [15]:
hexmap = choropleth_map(df_aggreg = df_aggreg, with_legend = True, kind = "filled_nulls") 

In [16]:
hexmap